In [ ]:
from konlpy.tag import Twitter
from gensim.models.word2vec import Word2Vec
from copy import deepcopy
import re
import os
from datetime import datetime
import csv
import sys 

t=Twitter()

TXT_PATH='./korea_txt/'
PDF_PATH='./korea_pdf/'
SAVE_PATH='./korea_wv/'

In [ ]:
def pdf_to_text(path):    #pdf 파일을 txt 파일로
    pdf_list=[]
    pdfs=os.listdir(PDF_PATH)
    for pdf in pdfs:
        ext = os.path.splitext(pdf)[-1]
        name=os.path.splitext(pdf)[-2]+'.txt'
        
        if ext=='.pdf':
            #pdf_list.append("%s%s" % (path, pdf))
            #pdf_name.append("%s"%pdf)
            os.system("python pdfminer.six-20170720/tools/pdf2txt.py -o %s%s %s%s" %(TXT_PATH, name, PDF_PATH, pdf))

def get_file(path): #txt파일 불러오기
    filelist=[]
    files = os.listdir(TXT_PATH) # txt파일에 있는 파일 리스트를 가져옴
    for file in files:
        ext = os.path.splitext(file)[-1]
            
        if ext == '.txt':
            filelist.append("%s%s" % (path, file))
    
    content=[]
    for file in filelist:
        content.append(preprocessing(open_file(file))) #content리스트 하나에 하나의 논문
    
    sentences = [tokenize(d) for d in content] #논문 하나가 한 리스트에 토큰화
    return sentences

def open_file(file): #txt 파일 오픈
    txt_file = open(file, encoding='utf-8')
    content = txt_file.read()
    txt_file.close()
    return content

def preprocessing(content): #전처리
    content = re.sub('\\xa0', '', content)
    content = re.sub('\\n', '', content)
    content = re.sub('\\\\xa0', '', content)
    content = re.sub('\\\\n', '', content)
    content = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@#$%&\\\=\(\'\")]', '', content)
    content = ' '.join(content.split())
                     
    return content

def tokenize(doc): #토크나이징
    return ['/'.join(t) for t in t.pos(doc, norm=True, stem=True)]

def wv_update(file): #새로운 논문 업데이트
    name=os.path.splitext(file)[-2]+'.txt'
    os.system("python pdfminer.six-20170720/tools/pdf2txt.py -o %s%s %s%s" %(TXT_PATH, name, PDF_PATH, file))
    
    new_token=[]
    content.append(preprocessing(open_file(TXT_PATH + name)))
    
    
    new_token=[tokenize(d) for d in content]
    
    wv_model_ko.build_vocab(new_token, update=True)
    wv_model_ko.train(new_token, total_examples=wv_model_ko.corpus_count, epochs=wv_model_ko.epochs)
    wv_model_ko.init_sims(replace=True)
    now=datetime.now()
    wv_model_ko.save('%s%s%s%s%s%s' % (SAVE_PATH ,now.year, now.month, now.day, now.hour, now.minute))

In [ ]:
%%time
#pdf_to_text(PDF_PATH) #pdf 파일 txt로 변환 실행

In [ ]:
%%time
sentences=get_file(TXT_PATH) #txt파일 read 후 토큰화

In [ ]:
sentences

In [ ]:
len(sentences)

In [ ]:
%%time
wv_model_ko = Word2Vec(sentences, size=100, window = 8, min_count=10, workers=4, iter=100, sg=1)
#100차원 벡터, 주변단어 2개 참조, 출연빈도 20번이상, CPU 쿼드코어, 100번 반복, CBOW, Skip-Gram중 후자
now=datetime.now()
wv_model_ko.save('%s%s%s%s%s%s' % (SAVE_PATH, now.year, now.month, now.day, now.hour, now.minute))

In [ ]:
wv_model_ko.corpus_count

In [ ]:
len(wv_model_ko.wv.vocab)

In [ ]:
wv_model_ko.wv.most_similar(tokenize('문학'), topn=5)

In [ ]:
wv_model_ko.wv.similarity(*tokenize('엄마 소녀'))

In [ ]:
wv_model_ko.predict_output_word(tokenize('프로테오믹스'))

In [ ]:
from matplotlib import font_manager, rc
font_fname='c:/windows/fonts/H2GTRM.TTF'
font_name=font_manager.FontProperties(fname=font_fname).get_name()
rc('font', family=font_name)

import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import nltk


from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
tsne_plot

In [ ]:
from matplotlib import font_manager, rc
font_fname='c:/windows/fonts/H2GTRM.TTF'
font_name=font_manager.FontProperties(fname=font_fname).get_name()
rc('font', family=font_name)

import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import nltk


from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

def display_closestwords_tsnescatterplot(model, word):
    
    arr = np.empty((0,100), dtype='f')
    word_labels = [word]

    # get close words
    close_words = model.similar_by_word(word)
    
    # add the vector for each of the closest words to the array
    arr = np.append(arr, np.array([model[word]]), axis=0)
    for wrd_score in close_words:
        wrd_vector = model[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)
        
    # find tsne coords for 2 dimensions
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)

    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    # display scatter plot
    plt.scatter(x_coords, y_coords)

    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    plt.xlim(x_coords.min()+0.00005, x_coords.max()+0.00005)
    plt.ylim(y_coords.min()+0.00005, y_coords.max()+0.00005)
    plt.show()

In [ ]:
%time
display_closestwords_tsnescatterplot(wv_model_ko, *tokenize('소설'))
#wv_model_bio.wv.most_similar(tokenize('유전자'))

In [ ]:
non=open('./txt/생식생물학에서_프로테오믹스의_응용.txt', encoding='utf-8').read()

content2=[]

content2.append(preprocessing(non)) #content리스트 하나에 하나의 논문

non_ko = [tokenize(d) for d in content2]  #논문 하나가 한 리스트에 토큰화

non_ko

In [ ]:
content2